In [1]:
import numpy as np
import cv2

import PIL.Image as Image
#import os

import matplotlib.pylab as plt

# import tensorflow as tf
#import tensorflow_hub as tensorflow_hub

from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.models import Sequential

import pathlib
import dlib

from sklearn.metrics import euclidean_distances
from tensorflow.keras.optimizers import Adam

In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../input/shape-predictor/shape_predictor_68_face_landmarks.dat')

In [3]:
confused_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Engaged/confused"
engaged_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Engaged/engaged"
frustrated_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Engaged/frustrated"

Lookingaway_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Not engaged/Looking Away"
bored_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Not engaged/bored"
drowsy_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Not engaged/drowsy"


confused_dataset_dir = pathlib.Path(confused_dataset_dir)
print(confused_dataset_dir)
engaged_dataset_dir = pathlib.Path(engaged_dataset_dir)
print(engaged_dataset_dir)
frustrated_dataset_dir = pathlib.Path(frustrated_dataset_dir)
print(frustrated_dataset_dir)
Lookingaway_dataset_dir = pathlib.Path(Lookingaway_dataset_dir)
print(Lookingaway_dataset_dir)
bored_dataset_dir = pathlib.Path(bored_dataset_dir)
print(bored_dataset_dir)
drowsy_dataset_dir = pathlib.Path(drowsy_dataset_dir)
print(drowsy_dataset_dir)

In [4]:
list(confused_dataset_dir.glob('*'))[:5]

In [5]:
image_count = len(list(confused_dataset_dir.glob('*')))
image_count

In [6]:
confused = list(confused_dataset_dir.glob("*"))
Image.open(str(confused[5])) 

In [7]:
image_dict = {
    'neutral': list(confused_dataset_dir.glob('*')),
    'engaged' : list(engaged_dataset_dir.glob('*')),
    'frustrated': list(frustrated_dataset_dir.glob('*')),
    'Lookingaway' : list(Lookingaway_dataset_dir.glob('*')),
    'bored': list(bored_dataset_dir.glob('*')),
    'drowsy' : list(drowsy_dataset_dir.glob('*')),
}

In [8]:
labels_dict = {
    'neutral': 0,
    'engaged' : 1,
    'frustrated': 2,
    'Lookingaway' : 3,
    'bored': 4,
    'drowsy' : 5,
}

labels_class = ['neutral','engaged','frustrated','Lookingaway','bored','drowsy',]

In [9]:
X = []
y = []

for label, images in image_dict.items():
  for image in images:
    img = cv2.imread(str(image))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    if len(rects) == 0:
      continue
    shape = predictor(gray, rects[0])

    shape_np = np.zeros((68, 2), dtype="int")

    for i in range(0, 68):
        shape_np[i] = (shape.part(i).x, shape.part(i).y)
    shape = shape_np

    eucl_dist = euclidean_distances(shape, shape)

    X.append(eucl_dist)
#     X.append(np.array(eucl_dist).flatten())

    y.append(labels_dict[label])

In [10]:
X = np.array(X)
y = np.array(y)

In [11]:
X.shape
#y.shape

In [12]:
# X_reshaped = X.reshape(3,-1)

In [13]:
# np.savetxt("xdata.csv", X_reshaped, delimiter=",")
# np.savetxt("ydata.csv", y, delimiter=",")

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [15]:
X_train.shape

In [16]:
import tensorflow as tf
X_train = tf.expand_dims(X_train, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)

In [17]:
X_train.shape

In [18]:
model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(68, 68)),  # input layer (1)
#     keras.layers.Dense(512, activation='sigmoid'),  # hidden layer (2)
#     keras.layers.Dense(512, activation='sigmoid'),  # hidden layer (3)
#     keras.layers.Dense(512, activation='sigmoid'),  # hidden layer (4)
#     keras.layers.Dense(512, activation='sigmoid'),  # hidden layer (5)
#     keras.layers.Dense(6, activation='softmax') # output layer (6)
    
    keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(68,68,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(68,68,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(6,activation='softmax')
])

In [19]:
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
             loss='sparse_categorical_crossentropy',
             metrics=["accuracy"])

model.summary()

In [20]:
history = model.fit(X_train,y_train,epochs = 100 , validation_split = 0.2)

In [21]:
model.save('model_landmark.h5')

In [22]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(100)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [23]:
model.evaluate(X_test,y_test)

In [24]:
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.metrics import roc_auc_score

In [25]:
predictions = model.predict(X_test)

In [26]:
predictions.shape

In [27]:
pred = []
for prediction in predictions:
    pred.append(np.argmax(prediction))

In [28]:
ruc = roc_auc_score(y_test, predictions,multi_class='ovr')
ruc

In [29]:
cm= confusion_matrix(y_test,pred)

In [30]:
import seaborn as sns 

In [31]:
f1=round(f1_score(y_test,pred,average='macro'),5)

In [32]:
sns.heatmap(cm,annot=True,xticklabels=labels_class,yticklabels=labels_class)
plt.title(f'F1 score: {f1}')
plt.show()

In [33]:
X_train_flatten = np.array(X_train).flatten()
X_train_flatten.reshape(1,-1)
X_train_flatten.shape

In [34]:
X_test_flatten = np.array(X_test).flatten()
X_test_flatten.reshape(1,-1)
X_test_flatten.shape

In [35]:
y_train.shape

In [36]:
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score 

In [ ]:
SupportVectorClassModel = SVC() 
SupportVectorClassModel.fit(X_train,y_train)

In [ ]:
SupportVectorClassModel.score(X_train,y_train)

In [ ]:
SupportVectorClassModel.score(X_test,y_test)